In [2]:
#packages needed to create form for user input 
import ipywidgets as widgets
from IPython.display import display

#packages related to dataframes
import pandas as pd
import numpy as np
from pathlib import Path

#package needed for regx purposes
import re


In [3]:
#Grabing the raw stock data and putting in dataframe
raw_stocks_data_to_load =  Path("../Data/Stock_Index_Raw_Data.csv")
raw_stocks_data_df = pd.read_csv(raw_stocks_data_to_load, header=0, parse_dates=True)
raw_stocks_data_df.sort_index(ascending = True, inplace = True)
raw_stocks_data_df.head()

,Date,Company,Ticker,Close_Price,Industry,Sector
0,2/1/2023,Apple,AAPL,145.43,Computer Manufacturing,Technology
1,2/2/2023,Apple,AAPL,150.82,Computer Manufacturing,Technology
2,2/3/2023,Apple,AAPL,154.50,Computer Manufacturing,Technology
3,2/6/2023,Apple,AAPL,151.73,Computer Manufacturing,Technology
4,2/7/2023,Apple,AAPL,154.65,Computer Manufacturing,Technology


## Creating Stock Drop Down Options 

In [4]:
#create new dataframe to only grab the unique values by company and only grabbing the columns needed 
dropdown_df = raw_stocks_data_df.drop_duplicates('Company').reset_index()
dropdown_df = dropdown_df[['Company', 'Ticker', 'Industry']]

#display new dataframe
dropdown_df

,Company,Ticker,Industry
0,Apple,AAPL,Computer Manufacturing
1,Microsoft,MSFT,Computer Software: Prepackaged Software
2,Amazon,AMZN,Catalog/Specialty Distribution
3,Nvidia,NVDA,Semiconductors
4,Alphabet,GOOGL,"Computer Software: Programming, Data Processing"
5,Meta,META,"Computer Software: Programming, Data Processing"
6,Tesla,TSLA,Auto Manufacturing
7,Berkshire Hathaway,BRK.B,Conglomerate
8,Unitedhealth Group,UNH,Medical Specialities
9,Jpmorgan Chase & Co,JPM,Major Banks


In [5]:
#creating a array to store the values for the dropdown 
dropdown_options_array = ['']

# Loop through the DataFrame and concat the display names of the dropdown options and push into array
for index, row in dropdown_df.iterrows():
    dropdown_value = row['Company'] + ' ('+ row['Ticker'] + ')' +' - ' + row['Industry']
    dropdown_options_array.append(dropdown_value)
    
# remove the exchange index from list  
chars_to_remove = ['(SPX)', '(NDAQ)']
dropdown_options_array = [item for item in dropdown_options_array if not any(char in item for char in chars_to_remove)]

#display dropdown_options array 
display(dropdown_options_array)

['',
 'Apple (AAPL) - Computer Manufacturing',
 'Microsoft (MSFT) - Computer Software: Prepackaged Software',
 'Amazon (AMZN) - Catalog/Specialty Distribution',
 'Nvidia (NVDA) - Semiconductors',
 'Alphabet (GOOGL) - Computer Software: Programming, Data Processing',
 'Meta (META) - Computer Software: Programming, Data Processing',
 'Tesla (TSLA) - Auto Manufacturing',
 'Berkshire Hathaway (BRK.B) - Conglomerate',
 'Unitedhealth Group (UNH) - Medical Specialities',
 'Jpmorgan Chase & Co (JPM) - Major Banks',
 'Johnson & Johnson (JNJ) - Biotechnology: Pharmaceutical Preparations',
 'Exxon Mobil (XOM) - Integrated oil Companies',
 'Visa (V) - Business Services',
 'Broadcom (AVGO) - Semiconductors']

# User Input for Game of Stocks

In [6]:
#input box for name
input_name = widgets.Textarea(
    value='Input your name',
    placeholder='Type something',
    description='First Name:',
    disabled=False
)


#Ask user how many stocks they would like in portfolio 

dropdown_widget_num_of_stocks = widgets.Dropdown(
    options=[1,2,3,4,5,6,7,8,9,10,"Choose Stock Amount:"],
    description="Amount", 
    value="Choose Stock Amount:")



#display input name, dropdown and instructions
print("""Welcome to the Game of Stocks! You will be asked a series of questions to get started. 
You can choose up to 10 stickes to create portfolio. Remember to diversify your portfolio.   """)

display(input_name)
# print(f"Choose number of stocks for your portfolio")
display(dropdown_widget_num_of_stocks)

# Create a button to trigger an action
button_stock_amount = widgets.Button(description="Submit")

# Define a function to be called on button click
def on_button_click_stock_amount(b):
    #declare and make it a global variable so it can be use in other places
    global name, num_of_stocks_choosen
    name = input_name.value
    num_of_stocks_choosen = dropdown_widget_num_of_stocks.value
    print(f"{name}, you have choosen stock amount: {num_of_stocks_choosen}")

# Attach the function to the button's click event
button_stock_amount.on_click(on_button_click_stock_amount)

# Display the button
display(button_stock_amount)


Welcome to the Game of Stocks! You will be asked a series of questions to get started. 
You can choose up to 10 stickes to create portfolio. Remember to diversify your portfolio.   


Textarea(value='Input your name', description='First Name:', placeholder='Type something')

Dropdown(description='Amount', index=10, options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 'Choose Stock Amount:'), valu…

Button(description='Submit', style=ButtonStyle())

James, you have choosen stock amount: 4


In [7]:
# Step 1: Create options for dropdown values which was done above

# Step 2: Create a for loop to generate the ipywidgets.Dropdown widgets and append them to the dropdown_widgets array
dropdown_widgets = []
for index in range(num_of_stocks_choosen):  # Passing user choosen number of stocks
    dropdown_widget = widgets.Dropdown(options=dropdown_options_array, description=f'Stock {index + 1}:')
    dropdown_widgets.append(dropdown_widget)
    
# Step 3: Create corresponding variables to store the selected values (same as before)
user_portfolio_array = [None] * len(dropdown_widgets)


# Step 4: Define the function for the submit button's on_click event (same as before)
def on_submit_button_click(b):
    for i, dropdown in enumerate(dropdown_widgets):
        # Use regular expression to find characters between parentheses
        pattern = r'\((.*?)\)'
        ticker_match = re.findall(pattern, dropdown.value) 
        user_portfolio_array[i]  = ''.join(ticker_match)
    print("Selected stocks:", user_portfolio_array)

# Create the Submit button (same as before)
submit_button = widgets.Button(description="Submit")
submit_button.on_click(on_submit_button_click)

# Display the dropdown widgets and the Submit button (same as before)
display(*dropdown_widgets, submit_button)



Dropdown(description='Stock 1:', options=('', 'Apple (AAPL) - Computer Manufacturing', 'Microsoft (MSFT) - Com…

Dropdown(description='Stock 2:', options=('', 'Apple (AAPL) - Computer Manufacturing', 'Microsoft (MSFT) - Com…

Dropdown(description='Stock 3:', options=('', 'Apple (AAPL) - Computer Manufacturing', 'Microsoft (MSFT) - Com…

Dropdown(description='Stock 4:', options=('', 'Apple (AAPL) - Computer Manufacturing', 'Microsoft (MSFT) - Com…

Button(description='Submit', style=ButtonStyle())

Selected stocks: ['AAPL', 'NVDA', 'META', 'XOM']


In [19]:
#calculating individual weights since we will assume it will be evenly split
weight = round((1/len(user_portfolio_array)),5)
weight

0.25

In [20]:
#creating the weight array to apply to 
weight_array = [weight] * len(user_portfolio_array)

In [22]:
weight_array

[0.25, 0.25, 0.25, 0.25]